In [5]:
# Tip: By now, install transformers from source
from transformers import AutoModelWithLMHead, AutoTokenizer
import pandas as pd 

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-question-generation-ap")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [2]:
def get_question(answer, context, max_length=64):
    input_text = "answer: %s  context: %s </s>" % (answer, context)
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                  attention_mask=features['attention_mask'],
                  max_length=max_length)

    return tokenizer.decode(output[0])

In [3]:
def qa_generation(context):
    print("context: " + context)
    answer = input("answer: ")
    print(answer)
    question = get_question(answer, context)[16:-4]
    print("question: " + question)
    add_question = input("add some word in questions : ")
    difficulty = input("difficulty(e,m,h,f): ")
    print(difficulty + '\n')

    return answer, question + '' + add_question, difficulty.upper()

In [6]:
def sentence_extract(dir):
    document = [] 
    with open("documents"+dir, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line != '\n' and line[0] != '=':
                sentence = line.strip('\n').split(". ")
                document.extend(sentence)
    
    return document

def sentence_extract2(dir):
    document = [] 
    with open("documents"+dir, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line != '\n' and line[0] != '=':
                sentence = line.strip('\n').split(". ")
                document.extend(sentence)

    two_sentence = ''
    new_document = []
    for i in document:
        if two_sentence == '':
            two_sentence = i
        else:
            new_document.append(two_sentence + '. ' + i + '.')
            two_sentence = ''
    
    return new_document

def sentence_extract3(dir):
    document = [] 
    with open("documents"+dir, "r") as file:
        while True:
            line = file.readline()
            if not line:
                break
            if line != '\n' and line[0] != '=':
                sentence = line.strip('\n').split(". ")
                document.extend(sentence)

    num_sentence = 0
    sentence = ''
    new_document = []
    for i in document:
        if num_sentence == 0:
            sentence = i
            num_sentence += 1
        elif num_sentence < 2:
            sentence = sentence + '. ' + i
            num_sentence += 1
        else:
            new_document.append(sentence + '. ' + i)
            sentence = ''
            num_sentence = 0
    
    return new_document
dir = "/pokemon_characters/eevee_pokemon.txt"
document = sentence_extract3(dir)

In [7]:
len(document)

25

In [9]:
answers = []
questions = []
documents = []
difficulties = []
for i, sentence in enumerate(document):
    if i < 20:
        continue
    elif i == 22:
        break
    a, q, d = qa_generation(sentence)

    if d =='F':
        continue
        
    answers.append(a)
    questions.append(q)
    difficulties.append(d) # E, M, H

context: Author Loredana Lipperini noted Eevee as being one of the "most mysterious Pokémon in the series". Technology Tell writer Jenni Lada noted that Eevee is a character "whom should feature" in Super Smash Bros., citing its customization "potential". Liz Finnegan of The Escapist listed Eevee as their eleventh favorite Pokémon, stating that Eevee is like the evolutionary equivalent of the turducken
yes
question: Is Eevee a good character for the Super Smash Bros.?
e

context: Gita Jackson of Kotaku claimed that Eevee is what Pokémon is all about, stating that "Eevee is adorable, but that’s not the only reason why I love them. Somewhere between a cat and a dog, Eevee seems more domesticated than longtime series mascot Pikachu, but still as mischievous as any other wild animal."In 2015, Eevee was the most traded Pokémon in the games' "Wonder Trade" feature. A special Pokémon Omega Ruby and Alpha Sapphire Online Competition known as the "Eevee Friendly Match" limits participants' Poké

In [51]:
dict = {'document_id': dir, 'question': questions, 'answer': answers, 'difficulty': difficulties}  
df = pd.DataFrame(dict) 
df.to_csv('test.csv', index=False)  
#df.to_csv('test.tsv', index=False, sep="\t") 

In [52]:
df1 = pd.read_csv ('result_4.csv')

In [53]:
df1['difficulty'].value_counts()

M    40
E    30
H    30
Name: difficulty, dtype: int64